In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv("insurance_pre.csv")

In [53]:
print(dataset.head())

   age     bmi  children      charges  sex_male  smoker_yes
0   19  27.900         0  16884.92400         0           1
1   18  33.770         1   1725.55230         1           0
2   28  33.000         3   4449.46200         1           0
3   33  22.705         0  21984.47061         1           0
4   32  28.880         0   3866.85520         1           0


In [4]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [5]:
dataset.head()

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0


In [6]:
x=dataset.drop(['charges'],axis=1) # axis=0: Drop rows ; axis=1: Drop columns.

In [7]:
x.head()

,age,bmi,children,sex_male,smoker_yes
0,19,27.900,0,0,1
1,18,33.770,1,1,0
2,28,33.000,3,1,0
3,33,22.705,0,1,0
4,32,28.880,0,1,0


In [8]:
y=dataset['charges']

In [9]:
y.head()

0    16884.92400
1     1725.55230
2     4449.46200
3    21984.47061
4     3866.85520
Name: charges, dtype: float64

In [19]:
from sklearn.model_selection import train_test_split

In [21]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=0)

In [69]:
pip install --upgrade lightgbm

Note: you may need to restart the kernel to use updated packages.


In [67]:
print(lgb.__version__)

4.6.0


In [93]:
import lightgbm as lgb
#model = lgb.LGBMRegressor()
model = lgb.LGBMRegressor(
    boosting_type='gbdt',
    num_leaves=31,
    max_depth=-1,
    learning_rate=0.01,
    n_estimators=1000,
    subsample_for_bin=200000,
    objective='regression',
    min_split_gain=0,
    min_child_weight=1e-3,
    min_child_samples=20,
    subsample=1.0,
    subsample_freq=0,
    colsample_bytree=1.0,
    reg_alpha=0.0,
    reg_lambda=0.0,
    random_state=42,
    n_jobs=-1
)
model.fit(
    x_train,
    y_train,
    eval_set=[(x_test, y_test)],
    eval_metric='rmse',
    callbacks=[
        lgb.early_stopping(stopping_rounds=100),
        lgb.log_evaluation(period=10)
    ]
)
y_pred = model.predict(x_test,num_iteration=model.best_iteration_)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001061 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 313
[LightGBM] [Info] Number of data points in the train set: 936, number of used features: 5
[LightGBM] [Info] Start training from score 13232.916456
Training until validation scores don't improve for 100 rounds
[10]	valid_0's rmse: 11541.9	valid_0's l2: 1.33216e+08
[20]	valid_0's rmse: 10575.5	valid_0's l2: 1.11842e+08
[30]	valid_0's rmse: 9718.89	valid_0's l2: 9.44568e+07
[40]	valid_0's rmse: 8959.39	valid_0's l2: 8.02706e+07
[50]	valid_0's rmse: 8287.9	valid_0's l2: 6.86893e+07
[60]	valid_0's rmse: 7696.1	valid_0's l2: 5.923e+07
[70]	valid_0's rmse: 7178.95	valid_0's l2: 5.15374e+07
[80]	valid_0's rmse: 6731.32	valid_0's l2: 4.53107e+07
[90]	valid_0's rmse: 6338.95	valid_0's l2: 4.01823e+07
[100]	valid_0's rmse: 5998.55	valid_0's l2: 3.59826e+07
[110]	valid_0's rmse: 5706.55	valid_0's l2: 3.256

In [94]:
from sklearn.metrics import mean_squared_error
# Evaluate the model
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"RMSE: {rmse:.4f}")

RMSE: 4332.1611


C:\Anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [95]:
from sklearn.metrics import r2_score

In [99]:
r_score=r2_score(y_test,y_pred)

In [101]:
r_score

0.8823121401738534